# Python for Linguists

Notebook 9: Chunking and Parsing

Venelin Kovatchev

University of Barcelona 2020

In this notebook we will see how we can do syntactic processing using NLTK. 

We will see two different approaches to syntax - chunking and full syntactic parsing.

In [1]:
# Import nltk
import nltk

In [2]:
# Download the important packages for today
nltk.download("conll2000")

[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Venelin\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [3]:
# Import corps for chunking
from nltk.corpus import conll2000



(Why) Text structure:

- “The meaning of a complex expression is determined by the meanings of its constituent expressions and **the rules used to combine them**”
    - Formal sentence structure, no semantics
- “A dog bites a man” vs “A man bites a dog” – in a bag-of-words, these are the same
- Compositional and non-compositional elements: “A dog bites John Smith”

Parsing and generation:
- “A Dog bites a man” -> who did what to whom (is the sentence possible at all)
- “Dog”, “bite”, “man” -> what kind of sentences can be generated using these words

Shallow parsing (chunking):
- Linear I-O-B structure. 
- No hierarchy. No (complex) long distance dependencies.
- Identify constituents.
- More computationally effective and higher accuracy.
- Can use a familiar approach (HMM)

In [4]:
# Split the conll corpus into test and train
test_sents = conll2000.chunked_sents('test.txt')
train_sents = conll2000.chunked_sents('train.txt')

# See the format of the chunked corpus
# Print the original format
print("Chunked sentence: {}".format(train_sents[99]))
# Print the I-O-B format
print("Chunked sentence in I-O-B format: {}".format(nltk.chunk.tree2conlltags(train_sents[99])))
# Draw a tree
# The tree will appear in a separate window
train_sents[99].draw()

Chunked sentence: (S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)
Chunked sentence in I-O-B format: [('Over', 'IN', 'B-PP'), ('a', 'DT', 'B-NP'), ('cup', 'NN', 'I-NP'), ('of', 'IN', 'B-PP'), ('coffee', 'NN', 'B-NP'), (',', ',', 'O'), ('Mr.', 'NNP', 'B-NP'), ('Stone', 'NNP', 'I-NP'), ('told', 'VBD', 'B-VP'), ('his', 'PRP$', 'B-NP'), ('story', 'NN', 'I-NP'), ('.', '.', 'O')]


(Full) Syntactic parsing:
- Assign a (hierarchical binary tree) structure to a sentence, given a grammar
- Input: Grammar (pre-defined), sentence
- Output: all possible trees (if any)

Syntactic ambiguity (multiple parses for the same sentence)
- Disambiguation through probabilistic grammers
- Rule based (heuristic) disambiguation
- Use of external resources (such as dictionaries, knowledge bases, and parsed corpora)

Context free grammar:
- sets of rules each of which expresses the ways the symbols of the language can be grouped and ordered together 
- a lexicon of words and symbols


- terminal nodes – the lexicon of the language (actual words)
- non-terminal nodes – generalization nodes (classes, such as POS)
- start symbol (S)
- derivation – a sequence of rule expansion (left to right)

In [5]:
# A simple CFG
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

# Test sentence
sent = "Mary saw Bob".split()

# Parse the sentence using the grammar
rd_parser = nltk.RecursiveDescentParser(grammar1)

# Print all possible trees
for tree in rd_parser.parse(sent):
    print(tree)

# Draw all possible trees
for tree in rd_parser.parse(sent):
    tree.draw()

(S (NP Mary) (VP (V saw) (NP Bob)))


In [ ]:
# Observe the workings of the different parser algorithms

# Top-down parser
# - Start from “s”
# - Generate a tree
# - Map the tree to the terminal nodes

nltk.app.rdparser()

In [ ]:
# Bottom-up parser
# - Start from the terminal nodes.
# - Group them into phrases.
# - Try to build up until S.
nltk.app.srparser()

In [ ]:
# Task 1
# use grammar1 to parse each sentence in the following corpus:

corpus = [['a', 'young', 'woman', 'walks', 'in', 'the', 'park'], 
['two', 'young', 'men', 'smile'], 
['a', 'young', 'woman', 'sees', 'two', 'men'], 
['sees', 'two', 'men', 'a', 'young', 'woman'], 
['a', 'young', 'woman', 'sees', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope'], 
['a', 'young', 'woman', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope', 'sees'], 
['two', 'angry', 'men', 'chase', 'a', 'woman', 'with', 'a', 'telescope'], 
['a', 'woman', 'I', 'know', 'owns', 'a', 'telescope'], 
['a', 'woman', 'I', 'know', 'a', 'telescope']]

In [ ]:
# Task 2
# Expand grammar1 with additional rules, so that you can see multiple parses for each sentence
# You should obtain the following number of parses for the sentences in corpus 1:
# “a young woman walks in the park” <- 1 parse
# “two young men smile” <- 1 parse
# “a young woman sees two men” <- 1 parse
# “sees two men a young woman” <- 0 parses
# “a young woman sees two old men in the park with a telescope” <- AT LEAST 3 parses
# “a young woman two old men in the park with a telescope sees” <- 0 parses
# “two angry men chase a woman with a telescope” <- 2 parses
# “a woman I know owns a telescope” <- 1 parse
# “a woman I know a telescope” <- 0 parses